In [1]:
import numpy as np
import pandas as pd
import scipy as sp

from statsmodels.stats.multitest import fdrcorrection

import cobra, cobra_utils

# Example of using reporter metabolites in *E. coli*

## Data Location

In [2]:
model_filename = '../data/e_coli_iML1515.xml'

In [3]:
gene_expression_filename = '../data/EcoliExpression_GSE54900.xlsx'

## Load data

**Metabolic model**

In [4]:
model = cobra_utils.io.load_model(model_filename, format='sbml')

Loading genome-scale model
Model correctly loaded.


**Gene expression**

In [5]:
gene_expression = pd.read_excel(gene_expression_filename, index_col='Gene')

In [6]:
gene_expression.head()

,wt_dpd_1,wt_dpd_2,delfur_dpd_1,delfur_dpd_2
Gene,,,,
b0002,8.113347,8.240386,6.915225,7.133376
b0003,7.918491,8.181859,6.652128,7.064316
b0004,7.248520,7.471647,6.059338,6.295427
b0005,3.758424,3.698816,3.207263,3.296815
b0006,4.696611,4.701444,4.635023,4.755118


## Get differential expession for two conditions

**p-value of differential expression**

In [7]:
gene_expression['p-value'] = gene_expression.apply(lambda row: sp.stats.ttest_ind(row[0:2], row[2:4])[1], axis=1)

**FDR**

In [8]:
rej, adj_p = fdrcorrection(gene_expression['p-value'].fillna(1.).values, alpha=0.05, is_sorted=False)

In [9]:
gene_expression['corrected p-value'] = adj_p

In [10]:
gene_expression.head()

,wt_dpd_1,wt_dpd_2,delfur_dpd_1,delfur_dpd_2,p-value,corrected p-value
Gene,,,,,,
b0002,8.113347,8.240386,6.915225,7.133376,0.011782,0.402468
b0003,7.918491,8.181859,6.652128,7.064316,0.039616,0.402468
b0004,7.248520,7.471647,6.059338,6.295427,0.018343,0.402468
b0005,3.758424,3.698816,3.207263,3.296815,0.012499,0.402468
b0006,4.696611,4.701444,4.635023,4.755118,0.953493,0.986424


## Reporter metabolites

**Compute reporter metabolites**

In [11]:
rep_met = cobra_utils.topology.reporter_metabolites(model, gene_expression[['corrected p-value']])

Running reporter metabolites analysis
Getting information for all metabolites in the model.
Information correctly obtained.


In [12]:
rep_met.head(20)

,p-value,corrected Z,mean Z,std Z,gene number
h_c,0.001037,3.079472,-0.524420,1.097564,885.0
glu__L_c,0.003624,2.685220,-0.195846,0.592217,56.0
h2o_c,0.003976,2.654081,-0.511701,0.952951,551.0
atp_c,0.004749,2.593602,-0.477499,1.026978,342.0
adp_c,0.005458,2.545394,-0.471624,1.056136,309.0
pi_c,0.019644,2.061160,-0.508153,1.066590,315.0
ptrc_p,0.023410,1.987931,-0.071768,0.429209,19.0
gln__L_c,0.026862,1.929057,-0.128925,0.401942,22.0
glc__D_p,0.029063,1.894749,-0.146143,0.681053,23.0
fadh2_c,0.031101,1.864859,-0.014573,0.338252,14.0


**Request reactions associated to reporter metabolites**

In [13]:
rxn_info = cobra_utils.query.rxn_info_from_metabolites(model, list(rep_met.head(16).index))

Using list of metabolites to get reactions where they participate. Also, getting genes of those reactions.
Information correctly obtained.


In [14]:
rxn_info

,RxnID,RxnName,GeneID,Subsystem,RxnFormula,MetID,MetName
0,CYTBO3_4pp,Cytochrome oxidase bo3 (ubiquinol-8: 4 protons...,b0431,,4.0 h_c + 0.5 o2_c + q8h2_c --> h2o_c + 4.0 h_...,h_c,H+
1,CYTBO3_4pp,Cytochrome oxidase bo3 (ubiquinol-8: 4 protons...,b0429,,4.0 h_c + 0.5 o2_c + q8h2_c --> h2o_c + 4.0 h_...,h_c,H+
2,CYTBO3_4pp,Cytochrome oxidase bo3 (ubiquinol-8: 4 protons...,b0432,,4.0 h_c + 0.5 o2_c + q8h2_c --> h2o_c + 4.0 h_...,h_c,H+
3,CYTBO3_4pp,Cytochrome oxidase bo3 (ubiquinol-8: 4 protons...,b0430,,4.0 h_c + 0.5 o2_c + q8h2_c --> h2o_c + 4.0 h_...,h_c,H+
4,NADS1,NAD synthase (nh3),b1740,,atp_c + dnad_c + nh4_c --> amp_c + h_c + nad_c...,h_c,H+
5,ACPPAT181,Acyl-(acyl carrier protein):phosphate acyltran...,b1090,,h_c + octeACP_c + pi_c --> ACP_c + ocdceap_c,h_c,H+
6,ACPPAT181,Acyl-(acyl carrier protein):phosphate acyltran...,b1094,,h_c + octeACP_c + pi_c --> ACP_c + ocdceap_c,h_c,H+
7,PGL,6-phosphogluconolactonase,b0767,,6pgl_c + h2o_c --> 6pgc_c + h_c,h_c,H+
8,WCOS,Tungsten pterin cofactor synthase,b0827,,2.0 h_c + mptamp_c + tungs_c --> amp_c + cu2_c...,h_c,H+
9,GDPDPK,GDP diphosphokinase,b2784,,atp_c + gdp_c --> amp_c + h_c + ppgpp_c,h_c,H+
